In [4]:
import pickle
import gzip
import numpy as np

# Load the dataset
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding = 'latin1')
f.close()

# gather intelligence on the data
print(type(train_set))
print(type(test_set))

<class 'tuple'>
<class 'tuple'>


In [5]:
#lets take a closer look now
print("Inputs:")
print(train_set[0])
print("Inputs shape is " + str(train_set[0].shape))
print("Input type is " + str(type(train_set[0])))
print("Labels:")
print(train_set[1])
print("Labels shape is" + str(train_set[1].shape))
print("Labels type is " + str(type(train_set[1])))


Inputs:
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Inputs shape is (50000, 784)
Input type is <class 'numpy.ndarray'>
Labels:
[5 0 4 ..., 8 4 8]
Labels shape is(50000,)
Labels type is <class 'numpy.ndarray'>


In [6]:
import numpy as np
def to_categorical(y, nb_classes):
    """ to_categorical.

    Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.

    Arguments:
        y: `array`. Class vector to convert.
        nb_classes: `int`. Total number of classes.

    """
    y = np.asarray(y, dtype='int32')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    Y[np.arange(len(y)),y] = 1.
    return Y

In [7]:
train_x = train_set[0]
train_y = to_categorical(train_set[1], 10)
test_x = test_set[0]
test_y = to_categorical(test_set[1],10)

#lets see result of categorical
print(test_y[:5])

[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [8]:
def generate_batch(batch_size):
    indexes = np.random.randint(49999, size = (batch_size,))
    return train_x[indexes], train_y[indexes]

In [9]:
import tensorflow as tf

In [10]:
tf.reset_default_graph() # resets all current tensorflow variables
#for members to fill in
x = tf.placeholder(tf.float32, [None, 784])
W = tf.get_variable("W", shape=[784, 10],
           initializer=tf.contrib.layers.xavier_initializer())
biases = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W)+biases)

In [11]:
y_ = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(.01).minimize(loss)

In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [13]:
for i in range(10000):
  batch_xs, batch_ys = generate_batch(32)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys}) 

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
#test your predictions, using built in test dataset
print(sess.run(accuracy, feed_dict={x: test_x, y_: test_y}))

0.9098
